A basic momentum strategy

In [3]:
import pandas as pd

from quantresearch.interface.binance import get_binance_close

# Parameters of our strategy
univ = ['BTCUSDT','ETHUSDT','ADAUSDT','BNBUSDT','XRPUSDT','DOTUSDT','MATICUSDT']
data_period = 4 # in hours
train_start = '2020-01-01'
train_end = '2022-12-31'
test_start = '2023-01-01'
test_end = '2023-12-31'

freq = f'{data_period}h'
train_prices = {x: get_binance_close(x, freq, start_ts=train_start, end_ts=train_end) for x in univ}
train_prices = pd.DataFrame.from_dict(train_prices, orient='columns')
train_prices = train_prices.reindex(pd.date_range(train_prices.index[0], train_prices.index[-1], freq=freq))
train_prices.head(3)

,BTCUSDT,ETHUSDT,ADAUSDT,BNBUSDT,XRPUSDT,DOTUSDT,MATICUSDT
2019-12-31 19:00:00,7230.71,130.18,0.03308,13.8159,0.19406,NaN,NaN
2019-12-31 23:00:00,7205.50,130.52,0.03320,13.7648,0.19518,NaN,NaN
2020-01-01 03:00:00,7195.80,130.84,0.03321,13.7162,0.19358,NaN,NaN


In [4]:
train_returns = train_prices.pct_change(fill_method=None)
train_returns.head(3)

,BTCUSDT,ETHUSDT,ADAUSDT,BNBUSDT,XRPUSDT,DOTUSDT,MATICUSDT
2019-12-31 19:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-12-31 23:00:00,-0.003487,0.002612,0.003628,-0.003699,0.005771,NaN,NaN
2020-01-01 03:00:00,-0.001346,0.002452,0.000301,-0.003531,-0.008198,NaN,NaN


In [6]:
from quantresearch.trading.algorithms.classic.momentum import rolling_avg_rank

# For each of the following multiples of data_period, we make a momentum strategy and calculate the strategy return:
horizons = [1, 2, 3, 4, 5, 6]
strats_returns = {}
for h in horizons:
    # Calculate the portfolio weights for the strategy
    weights = rolling_avg_rank(train_returns, h)
    # Calculate the return of the strategy for each period
    strats_returns[h] = (weights.shift()*train_returns).sum(axis=1)
strats_returns = pd.DataFrame(strats_returns)
strats_returns.head(5)

,1,2,3,4,5,6
2019-12-31 19:00:00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000
2019-12-31 23:00:00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000
2020-01-01 03:00:00,-1.281039e-03,-0.001281,-0.001281,-0.001281,-0.001281,-0.001281
2020-01-01 07:00:00,2.181741e-03,0.001558,0.001558,0.001558,0.001558,0.001558
2020-01-01 11:00:00,-7.816796e-07,0.000616,0.002451,0.002451,0.002451,0.002451


In [4]:
from quantresearch.trading.evaluation.metric import sharpe_ratio

# Calculate the annualized Sharpe of each strategy
sr = sharpe_ratio(strats_returns, data_period)
sr

1   -0.989473
2   -0.238533
3    0.259369
4    1.071280
5    1.490998
6    0.899934
dtype: float64

In [5]:
# Since 5*4=20 hours gained the highest sharpe ratio, we pick this value as the trained parameter of our strategy
horizon_trained = 20

# Then, we test this strategy against the test data
test_prices = {x: get_binance_close(x, freq, start_ts=test_start, end_ts=test_end) for x in univ}
test_prices = pd.DataFrame.from_dict(test_prices, orient='columns')
test_prices = test_prices.reindex(pd.date_range(test_prices.index[0], test_prices.index[-1], freq=freq))
test_prices

,BTCUSDT,ETHUSDT,ADAUSDT,BNBUSDT,XRPUSDT,DOTUSDT,MATICUSDT
2022-12-31 19:00:00,16521.36,1194.19,0.24522,245.1529,NaN,4.290,0.7530
2022-12-31 23:00:00,16528.14,1195.32,0.24514,243.9536,NaN,4.310,0.7520
2023-01-01 03:00:00,16550.51,1196.17,0.24586,244.2534,NaN,4.330,0.7550
2023-01-01 07:00:00,16557.02,1196.59,0.24610,244.3091,NaN,4.330,0.7560
2023-01-01 11:00:00,16599.38,1201.98,0.24852,245.0832,NaN,4.380,0.7620
...,...,...,...,...,...,...,...
2023-12-30 03:00:00,41899.99,2285.90,0.59790,316.5000,0.6202,8.195,0.9574
2023-12-30 07:00:00,42394.18,2313.94,0.61010,318.8000,0.6288,8.385,0.9724
2023-12-30 11:00:00,42360.00,2299.00,0.60780,315.6000,0.6254,8.338,0.9677
2023-12-30 15:00:00,42138.01,2293.40,0.60140,316.8000,0.6219,8.367,0.9511


In [6]:
test_returns = test_prices.pct_change(fill_method=None)
test_returns

,BTCUSDT,ETHUSDT,ADAUSDT,BNBUSDT,XRPUSDT,DOTUSDT,MATICUSDT
2022-12-31 19:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-12-31 23:00:00,0.000410,0.000946,-0.000326,-0.004892,NaN,0.004662,-0.001328
2023-01-01 03:00:00,0.001353,0.000711,0.002937,0.001229,NaN,0.004640,0.003989
2023-01-01 07:00:00,0.000393,0.000351,0.000976,0.000228,NaN,0.000000,0.001325
2023-01-01 11:00:00,0.002558,0.004504,0.009833,0.003169,NaN,0.011547,0.007937
...,...,...,...,...,...,...,...
2023-12-30 03:00:00,-0.000909,-0.004395,-0.007140,-0.005343,-0.003855,-0.015379,-0.011869
2023-12-30 07:00:00,0.011795,0.012267,0.020405,0.007267,0.013866,0.023185,0.015667
2023-12-30 11:00:00,-0.000806,-0.006457,-0.003770,-0.010038,-0.005407,-0.005605,-0.004833
2023-12-30 15:00:00,-0.005241,-0.002436,-0.010530,0.003802,-0.005596,0.003478,-0.017154


In [28]:
from quantresearch.trading.evaluation.metric import strategy_returns

# Apply the trained strategy to the test data
weights = rolling_avg_rank(test_returns, horizon_trained)
# Calculate the return of the strategy over each test period
strat_returns = strategy_returns(weights, test_returns)
strat_returns

2022-12-31 19:00:00    0.000000
2022-12-31 23:00:00    0.000000
2023-01-01 03:00:00    0.000313
2023-01-01 07:00:00    0.000131
2023-01-01 11:00:00    0.003304
                         ...   
2023-12-30 03:00:00    0.003509
2023-12-30 07:00:00   -0.003829
2023-12-30 11:00:00   -0.001690
2023-12-30 15:00:00    0.004224
2023-12-30 19:00:00    0.005529
Freq: 4h, Length: 2185, dtype: float64

In [30]:
sharpe = sharpe_ratio(strat_returns, data_period)
sharpe

np.float64(0.756299226388108)

In [9]:
strat_returns_cumulative = strat_returns.cumsum()
print(strat_returns_cumulative)

2022-12-31 19:00:00    0.000000
2022-12-31 23:00:00    0.000000
2023-01-01 03:00:00    0.000313
2023-01-01 07:00:00    0.000445
2023-01-01 11:00:00    0.003748
                         ...   
2023-12-30 03:00:00    0.114554
2023-12-30 07:00:00    0.110725
2023-12-30 11:00:00    0.109035
2023-12-30 15:00:00    0.113259
2023-12-30 19:00:00    0.118788
Freq: 4h, Length: 2185, dtype: float64


In [31]:
import importlib
import quantresearch.trading.evaluation.metric
importlib.reload(quantresearch.trading.evaluation.metric)

from quantresearch.trading.evaluation.metric import max_drawdown_stats

stats = max_drawdown_stats(strat_returns_cumulative)
stats

{'value': np.float64(-2.390973303713002),
 'peak_date': Timestamp('2023-01-08 19:00:00'),
 'trough_date': Timestamp('2023-11-30 15:00:00'),
 'recovery_date': Timestamp('2023-12-13 15:00:00'),
 'drawdown_series': 2022-12-31 19:00:00         NaN
 2022-12-31 23:00:00         NaN
 2023-01-01 03:00:00    0.000000
 2023-01-01 07:00:00    0.000000
 2023-01-01 11:00:00    0.000000
                          ...   
 2023-12-30 03:00:00   -0.110724
 2023-12-30 07:00:00   -0.140447
 2023-12-30 11:00:00   -0.153566
 2023-12-30 15:00:00   -0.120777
 2023-12-30 19:00:00   -0.077856
 Freq: 4h, Length: 2185, dtype: float64}